In [ ]:
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import os
from scipy.stats import ttest_ind, shapiro, levene
from sklearn.utils.extmath import randomized_svd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
from itertools import combinations
from hotelling.stats import hotelling_t2

In [ ]:
def calculate_and_store_SVD():
    df = pd.read_csv('data/filtered_df2.csv')
    image_files = list(df['Image']) 

    sample_img = Image.open("data/eyeball_img/" + image_files[0])
    M, N = sample_img.size
    print(f"Image Dimensions: {M}, {N}")

    images_array = np.zeros((len(image_files), M * N), dtype=np.float32)

    for i, file in enumerate(image_files):
        print(i)
        img = Image.open("data/eyeball_img/" + file).convert('RGB')
        img = ImageOps.grayscale(img)
        img_array = np.asarray(img, dtype=np.float32) / 255.0  # normalize to [0,1]
        images_array[i, :] = img_array.flatten()  # flatten

    # Perform randomized SVD
    U, S, Vt = randomized_svd(images_array, n_components=100, random_state=42)

    # No need to sort! randomized_svd already sorts singular values descending

    np.savez("grayscale_svd_results2.npz", U=U, S=S, Vt=Vt)

In [ ]:
condition_mapping = {
    'N': 'Normal',
    'D': 'Diabetes',
    'G': 'Glaucoma',
    'C': 'Cataract',
    'A': 'AMD',
    'H': 'Hypertension',
    'M': 'Myopia',
    'O': 'Other'
}

def get_condition(row):
    for col, label in condition_mapping.items():
        if row[col] == 1:
            return col
    return 'Unknown'

def plot_top_singular_values():
    data = np.load("grayscale_svd_results2.npz")
    U, S, Vt = data["U"], data["S"], data["Vt"]
    top_2_U = U[:, :20]
    top_2_S = S[:20]
    # top_2_coefficients = top_2_U * top_2_S
    top_2_coefficients = top_2_U
    print(np.shape(top_2_U))
    df = pd.read_csv('data/filtered_df2.csv')
    df['top_1_coefficient'] = top_2_coefficients[:, 18]  # First singular vector coefficient
    df['top_2_coefficient'] = top_2_coefficients[:, 19]  # Second singular vector coefficient
    display(df)

    colors = []
    for index, row in df.iterrows():
        if row['N'] == 1:
            colors.append('grey')
        elif row['D'] == 1:
            colors.append('red')
        elif row['G'] == 1:
            colors.append('yellow')
        elif row['C'] == 1:
            colors.append('green')
        elif row['A'] == 1:
            colors.append('orange')
        elif row['H'] == 1:
            colors.append('pink')
        elif row['M'] == 1:
            colors.append('blue')
        elif row['O'] == 1:
            colors.append('purple')
    df.plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c=colors, alpha = 0.2)
    plt.show()
    plt.clf()
    df[df['N'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='grey')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['D'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='red')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['G'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='yellow')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['C'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='green')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['A'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='orange')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['H'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='pink')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['M'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='blue')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()
    df[df['O'] == 1].plot.scatter(x='top_1_coefficient', y='top_2_coefficient', c='purple')
    plt.xlim(0, 0.05)
    plt.ylim(-0.15, 0.15)
    plt.show()

    indicators = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

    df['condition'] = df.apply(get_condition, axis=1)
    df.boxplot(column='top_1_coefficient', by='condition', grid=False)
    df.boxplot(column='top_2_coefficient', by='condition', grid=False)

    # create an 8x8 grid of subplots
    fig, axes = plt.subplots(8, 8, figsize=(20, 20))
    for i in range(8):
        for j in range(8):
            var1 = indicators[i]
            var2 = indicators[j]

            # points where var1 == 1 and var2 == 0 (first class)
            class_1 = df[(df[var1] == 1) & (df[var2] == 0)]
            # points where var1 == 0 and var2 == 1 (second class)
            class_2 = df[(df[var1] == 0) & (df[var2] == 1)]

            axes[i, j].scatter(class_1['top_1_coefficient'], class_1['top_2_coefficient'], color='red', label=var1, alpha=0.2)
            axes[i, j].scatter(class_2['top_1_coefficient'], class_2['top_2_coefficient'], color='blue', label=var2, alpha=0.2)
            
            axes[i, j].set_xlabel('Top Singular Vector 1')
            axes[i, j].set_ylabel('Top Singular Vector 2')
            axes[i, j].set_title(f'{var1} vs {var2}')

            axes[i, j].legend()

    plt.tight_layout()
    plt.show()
    plt.clf()

In [ ]:
#calculate_and_store_SVD()

In [ ]:
#plot_top_singular_values()

In [ ]:
# Notes:
# 1: G
# 2: None
# 3: C
# 4: None
# 5: None
# 6: None
# 7: None
# 8: C*
# 9: None
# 10: None
# 12: None
# 13: None
# 14: M
# 15: None
# 16: None
# 17: C
# 18: C
# 19: C
# 20: C

In [ ]:
df = pd.read_csv('data/filtered_df2.csv')
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_image_files = list(train_df['Image']) 



sample_img = Image.open("data/eyeball_img/" + train_image_files[0])
M, N = sample_img.size
print(f"Image Dimensions: {M}, {N}")

train_images_array = np.zeros((len(train_image_files), M * N), dtype=np.float32)

for i, file in enumerate(train_image_files):
    print(i)
    img = Image.open("data/eyeball_img/" + file).convert('RGB')
    img = ImageOps.grayscale(img)
    img_array = np.asarray(img, dtype=np.float32) / 255.0  # normalize to [0,1]
    train_images_array[i, :] = img_array.flatten()  # flatten

# Perform randomized SVD
U, S, Vt = randomized_svd(train_images_array, n_components=100, random_state=42)

test_image_files = list(test_df['Image']) 
test_images_array = np.zeros((len(test_image_files), M * N), dtype=np.float32)
for i, file in enumerate(test_image_files):
    print(i)
    img = Image.open("data/eyeball_img/" + file).convert('RGB')
    img = ImageOps.grayscale(img)
    img_array = np.asarray(img, dtype=np.float32) / 255.0  # normalize to [0,1]
    test_images_array[i, :] = img_array.flatten()  # flatten

print(test_df.shape)
n = train_df.shape[0]
X_train = U
y_train = train_df['C']
X_test = test_images_array @ Vt.T
y_test = test_df['C']

print(X_train.shape)
print(X_test.shape)

clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))